# Predección del semestre en cual un estudiante abandona sus estudios de pregrado

<img src="https://raw.githubusercontent.com/fhernanb/fhernanb.github.io/master/docs/logo_unal_color.png" alt="drawing" width="200"/>



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas import read_excel
from sklearn.model_selection import train_test_split

## Leyendo los datos

In [2]:
my_sheet = 'SinDuplicadosDepurada' # change it to your sheet name
file_name = 'BDFinal.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name=my_sheet)
df.shape

(6593, 64)

En la siguiente línea se crea la variable `y` categórica.

In [3]:
df['y'] = pd.Categorical(df['NIVEL_PREGRADO'])

In [4]:
pd.crosstab(index=df['y'], columns="Frecuencia")

col_0,Frecuencia
y,
1,4768
2,867
3,455
4,226
5,135
6,73
7,24
8,9
9,7


## Creación de los datos de entrenamiento (train) y de validación (test)
Para particionar los datos originales se usa la función `train_test_split`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df[['PROMEDIO_ULTIMO_SEMESTRE_TERMINADO',
                                                        'VERSION',
                                                        'CREDITOS_ULTIM_SEMEST_MATRIC',
                                                        'IND_POBREZA',
                                                        'Puntaje_total_estan',
                                                        'Puntaje_global',
                                                        'estu_edad']], 
                                                    df['y'], test_size=0.20)

## Ajuste del modelo SVM

Para crear el modelo se usa la función `sklearn.svm.SVC`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](https://scikit-learn.org/0.16/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [6]:
cw = {}
for i in set(y_train):
    cw[i] = np.sum(y_train == i)
print(cw)

{1: 3808, 2: 699, 3: 359, 4: 182, 5: 105, 6: 61, 7: 19, 8: 8, 9: 7, 10: 20, 11: 6}


In [7]:
from sklearn.svm import SVC

model = SVC(class_weight=cw, coef0=0.0, degree=3, gamma='auto', kernel='linear')
model_fit = model.fit(X=X_train, y=y_train)

Para obtener la tabla de confusión hacemos lo siguiente:

In [8]:
t1 = pd.crosstab(index=model.predict(X_train), columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])

print('\n')
print('La tabla de confusión usando train es: \n')
print(t1)

print('\n')
print('La tabla de confusión usando test es: \n')
print(t2)



La tabla de confusión usando train es: 

Membresia     1    2    3    4    5   6   7   8   9   10  11
Prediccion                                                  
1           3808  699  359  182  105  61  19   8   7  20   6


La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   10
Prediccion                                      
1           960  168  96  44  30  12   5   1   3


In [9]:
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))

El Accuracy con test es: 0.7220326128175958
El Accuracy con train es: 0.7278241091736164


## Ajuste del modelo KNN

In [10]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=10)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   10
Prediccion                                      
1           950  166  95  44  30  11   5   1   3
2             9    1   1   0   0   1   0   0   0
3             0    1   0   0   0   0   0   0   0
5             1    0   0   0   0   0   0   0   0


El Accuracy con test es: 0.7222222222222222
El Accuracy con train es: 0.7210007581501138


## Ajuste del modelo ARBOL

In [11]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   10
Prediccion                                      
1           722  102  57  26  11   2   4   0   1
2           121   29  23   8   5   2   1   0   0
3            50   19   7   5   3   2   0   0   0
4            24    9   4   3   7   4   0   0   0
5            23    3   2   1   0   1   0   0   0
6            13    5   3   1   2   1   0   1   2
7             2    0   0   0   0   0   0   0   0
8             2    0   0   0   0   0   0   0   0
9             1    0   0   0   0   0   0   0   0
10            2    1   0   0   2   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.577710386656558


## Ajuste del modelo Gradient Tree Boosting

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   10
Prediccion                                      
1           936  157  85  38  29   8   4   1   2
2             6    3   2   3   0   1   1   0   0
3             2    2   5   0   1   0   0   0   0
4             2    1   2   2   0   1   0   0   0
5             4    1   0   0   0   1   0   0   0
6             2    2   0   1   0   0   0   0   0
7             2    2   1   0   0   1   0   0   1
8             1    0   1   0   0   0   0   0   0
10            3    0   0   0   0   0   0   0   0
11            2    0   0   0   0   0   0   0   0


El Accuracy con test es: 0.7817595752749337
El Accuracy con train es: 0.7172100075815011


## Ajuste del modelo Forests of randomized trees¶

In [13]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1   2   3   4   5   6   7   8   10
Prediccion                                     
1           715  98  56  25  10   4   4   0   1
2           132  26  22   7   5   1   1   0   1
3            51  20   6   7   3   2   0   0   1
4            24  12   4   3   6   2   0   0   0
5            19   3   4   1   2   2   0   0   0
6            12   5   4   1   3   0   0   1   0
7             3   2   0   0   0   0   0   0   0
8             2   1   0   0   0   0   0   0   0
10            1   1   0   0   1   0   0   0   0
11            1   0   0   0   0   1   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.5701288855193328


## Ajuste del modelo Multioutput classification

In [22]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                      hidden_layer_sizes=(5, 5, 5), random_state=1)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   10
Prediccion                                      
1           960  168  96  44  30  12   5   1   3


El Accuracy con test es: 0.7220326128175958
El Accuracy con train es: 0.7278241091736164
